In [2]:
!pip install xgboost

You should consider upgrading via the 'C:\Users\user\anaconda3\python.exe -m pip install --upgrade pip' command.


In [18]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

import xgboost as xgb

In [19]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(15) # Seed 고정

In [20]:
# Data Load

train_df = pd.read_csv('train.csv')


In [21]:
train_x = train_df.filter(regex='X') # Input : X Featrue : 56
train_y = train_df.filter(regex='Y') # Output : Y Feature : 14

In [22]:
# 피처 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_data_scaled = scaler.fit_transform(train_x)

X_data_scaled.shape

(39607, 56)

In [23]:
# {'activation': 'relu', 'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 3, 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'adam', 'warm_start': False}

#learning_rate = 0.01 에서 0.0003 으로 바꿈


In [24]:
# MLPRegressor

from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(2,20)],
              'activation': ['LeakyReLU', 'relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.0003],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [True],
              'warm_start': [False]}
mlp_GS = GridSearchCV(mlp, param_grid=param_grid, 
                   cv=10, verbose=True, pre_dispatch='2*n_jobs')
mlp_GS.fit(train_x, train_y)


Fitting 10 folds for each of 36 candidates, totalling 360 fits


c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 673, in fit
    return self._fit(X, y, incremental=False)
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 357, in _fit
    self._validate_hyperparameters()
  File "c:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 448, in _validate_hyperparameters
    raise ValueError("The activation '%s' is not supported. Supported "
ValueError: The activation 'LeakyReLU' is not su

GridSearchCV(cv=10, estimator=MLPRegressor(),
             param_grid={'activation': ['LeakyReLU', 'relu'], 'alpha': [0.0001],
                         'early_stopping': [True],
                         'hidden_layer_sizes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                                12, 13, 14, 15, 16, 17, 18,
                                                19],
                         'learning_rate': ['constant'],
                         'learning_rate_init': [0.0003], 'max_iter': [1000],
                         'power_t': [0.5], 'solver': ['adam'],
                         'warm_start': [False]},
             verbose=True)

In [25]:
print(mlp_GS.best_params_)

{'activation': 'relu', 'alpha': 0.0001, 'early_stopping': True, 'hidden_layer_sizes': 3, 'learning_rate': 'constant', 'learning_rate_init': 0.0003, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'adam', 'warm_start': False}


In [26]:
test_x = pd.read_csv('test.csv').drop(columns=['ID'])

In [27]:
preds = mlp_GS.predict(test_x)
print('Done.')

Done.


In [28]:
submit = pd.read_csv('sample_submission.csv')

In [29]:
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')

Done.


In [30]:
submit.to_csv('mlp.csv', index=False)